In [16]:
# Created on Thu Dec 20 13:23:15 2018
#
# @author: espenfb

In [17]:
import os
import sys
from pathlib import Path

filepath = os.path.abspath('../')
sys.path.append(str(Path(filepath) / "src"))

import detInvModel as dim
import pandas as pd
pd.set_option("plotting.backend","plotly")
import savedRes as sr

In [18]:
time_data = {
    'start_date': pd.Timestamp(year=2019, month=1, day=1),
    'end_date': pd.Timestamp(year=2020, month=1, day=1),
    'ref_date': pd.Timestamp(year=2019, month=1, day=1)}
dirs = {
    'data_dir': "data\\",
    'ctrl_data_file': 'ctrl_data.csv',
    'res_dir': 'results\\'}


In [19]:
obj = dim.deterministicModel(time_data, dirs)

0 load Series/load_series.csv
data\Series/load_series.csv
1 wind Series/pot_wind_profiles.csv
data\Series/pot_wind_profiles.csv
2 inst_wind Series/inst_wind_profiles.csv
data\Series/inst_wind_profiles.csv
4 solar Series/solar_profiles.csv
data\Series/solar_profiles.csv
5 offshore_wind Series/offshore_wind_series.csv
data\Series/offshore_wind_series.csv
6 inflow_series Series/inflow_series.csv
data\Series/inflow_series.csv
7 inflow_series_ureg Series/inflow_series_ureg.csv
data\Series/inflow_series_ureg.csv
8 import_price Series/import_price_utc.csv
data\Series/import_price_utc.csv
9 export_price Series/export_price_utc.csv
data\Series/export_price_utc.csv


In [20]:
import copy
di = {}
# SET
di['TIME'] = {None: list(obj.timerange)}
di['LAST_TIME'] = {None: [list(obj.timerange)[-1]]}

# Set node structure
node_data = copy.copy(obj.data.bus)
el_pf = 'EN'
h2_pf = 'HN'
market_pf = "MN"

dual_buses = node_data[node_data.Type == "Dual"]

di['EL_NODES'] = {None: [el_pf + '%.2d' %
                            i for i in dual_buses.Bus.tolist()]}
di['H2_NODES'] = {None: [h2_pf + '%.2d' %
                            i for i in dual_buses.Bus.tolist()]}

di['INTERNAL_NODES'] = {None: di['EL_NODES'][None] + di['H2_NODES'][None]}

market_buses = node_data[node_data.Type == "Market"]
market_buses_nr = market_buses.Bus.tolist()
di['MARKET_NODES'] = {None: [market_pf + '%.2d' %
                                i for i in market_buses.Bus.tolist()]}

di['NODES'] = {None: di['INTERNAL_NODES'][None] + di['MARKET_NODES'][None]}


In [21]:
def get_prefix(n, marked_buses, default=el_pf):
    if n in marked_buses:
        return market_pf
    else:
        return default

# Set line structure
line_data = copy.copy(obj.data.line)
mb_str = ['%.2d' % i for i in market_buses_nr]
branch_indx = []
new_branch_indx = []
pipe_indx = []
for i in line_data.index:
    from_bus = '%.2d' % line_data.From[i]
    to_bus = '%.2d' % line_data.To[i]
    if line_data.Type[i] == 'Existing':
        from_typ = get_prefix(from_bus, mb_str)
        to_typ = get_prefix(to_bus, mb_str)
        branch_indx.append((i, from_typ + from_bus, to_typ + to_bus))
    elif line_data.Type[i] == 'New':
        from_typ = get_prefix(from_bus, mb_str)
        to_typ = get_prefix(to_bus, mb_str)
        new_branch_indx.append((i, from_typ + from_bus, to_typ + to_bus))
    elif line_data.Type[i] == 'H2':
        from_typ = get_prefix(from_bus, mb_str, default=h2_pf)
        to_typ = get_prefix(to_bus, mb_str, default=h2_pf)
        if (from_typ != market_pf) or (to_typ != market_pf):
            pipe_indx.append((i, from_typ + from_bus, to_typ + to_bus))


In [22]:
branch_indx


[(0, 'EN01', 'EN02'),
 (1, 'EN01', 'EN03'),
 (2, 'EN03', 'EN04'),
 (3, 'EN01', 'EN05'),
 (4, 'EN04', 'EN05'),
 (5, 'EN05', 'EN06'),
 (6, 'EN06', 'EN07'),
 (7, 'EN04', 'MN09'),
 (8, 'EN06', 'MN10'),
 (9, 'EN06', 'EN08'),
 (10, 'EN08', 'MN11'),
 (11, 'EN05', 'MN12')]

In [23]:
new_branch_indx


[(12, 'EN01', 'EN02'),
 (13, 'EN01', 'EN03'),
 (14, 'EN03', 'EN04'),
 (15, 'EN01', 'EN05'),
 (16, 'EN04', 'EN05'),
 (17, 'EN05', 'EN06'),
 (18, 'EN06', 'EN07'),
 (19, 'EN04', 'MN09'),
 (20, 'EN06', 'MN10'),
 (21, 'EN06', 'EN08'),
 (22, 'EN08', 'MN11'),
 (23, 'EN01', 'EN13'),
 (24, 'EN05', 'MN12')]

In [24]:
pipe_indx


[(25, 'HN01', 'HN02'),
 (26, 'HN01', 'HN03'),
 (27, 'HN03', 'HN04'),
 (28, 'HN01', 'HN05'),
 (29, 'HN04', 'HN05'),
 (30, 'HN05', 'HN06'),
 (31, 'HN06', 'HN07'),
 (32, 'HN06', 'HN08'),
 (33, 'HN01', 'HN13')]

In [25]:
di['MARKET_NODES']


{None: ['MN09', 'MN10', 'MN11', 'MN12']}

In [26]:
# Set line structure
line_data = copy.copy(obj.data.line)
branch_indx = []
new_branch_indx = []
pipe_indx = []
for i in line_data.index:
    from_bus = '%.2d' % line_data.From[i]
    to_bus = '%.2d' % line_data.To[i]
    if line_data.Type[i] == 'Existing':
        n_typ = el_pf
        branch_indx.append((i, n_typ + from_bus, n_typ + to_bus))
    elif line_data.Type[i] == 'New':
        n_typ = el_pf
        new_branch_indx.append((i, n_typ + from_bus, n_typ + to_bus))
    elif line_data.Type[i] == 'H2':
        n_typ = h2_pf
        pipe_indx.append((i, n_typ + from_bus, n_typ + to_bus))

di['CURRENT_LINES'] = {None: branch_indx}
di['NEW_LINES'] = {None: new_branch_indx}
di['LINES'] = {None: di['CURRENT_LINES'][None] + di['NEW_LINES'][None]}

di['PIPES'] = {None: pipe_indx}

di['BRANCHES'] = {None: di['LINES'][None] + di['PIPES'][None]}
di['NEW_BRANCHES'] = {None: di['NEW_LINES'][None] + di['PIPES'][None]}

def getBranchesAtNode(set_type):
    out = {}
    for node in di['NODES'][None]:
        for n, i, j in di[set_type][None]:
            if i == node or j == node:
                if node not in out.keys():
                    out[node] = []
                out[node].append((n, i, j))
    return out
di['BRANCHES_AT_NODE'] = getBranchesAtNode('BRANCHES')


In [27]:
bn = di['BRANCHES_AT_NODE']


In [28]:
bn.keys()

dict_keys(['EN01', 'EN02', 'EN03', 'EN04', 'EN05', 'EN06', 'EN07', 'EN08', 'EN13', 'HN01', 'HN02', 'HN03', 'HN04', 'HN05', 'HN06', 'HN07', 'HN08', 'HN13'])

In [29]:
node_data.Type

0       Dual
1       Dual
2       Dual
3       Dual
4       Dual
5       Dual
6       Dual
7       Dual
8     Market
9     Market
10    Market
11    Market
12      Dual
Name: Type, dtype: object

In [30]:
obj.buildModel()

Building deterministic investment model...
Creating LP problem instance...
ERROR: Constructing component 'minHydroProd' from data=None failed:
        KeyboardInterrupt:


KeyboardInterrupt: 

In [ ]:
obj.solve()

In [ ]:
obj.processResults()

In [ ]:
obj.saveRes(dirs['res_dir'])

res = sr.savedRes(dirs['res_dir'], data=obj.data)